# <center> TP2 - 01 Description

# Objective of TP

In this TP you will develop the full **supervised learning pipeline** including the *hyper-parameter tuning* and *model evalutaion*.  

You will then apply the pipeline to three algorithms
* nearest neighbour
* decision tree
* default classifier

Finally, you will perform *model comparison* and **discuss** its results.

### Recommendation:
The code you will develop in this TP is to be re-used in TP3 and the exam.  
Therefore we recommend you try to make it clear (use comments, when printing say what you print) so that next time it is easier for you to remember what it does.  
Also, try to make the code generic so that it can be easilly used for different datasets.   
Try to automate as much as possible so that the code does not require too much of your attention, finally you will need to do the same type of analysis not for 3 algorithms but for 5-6.

## Dataset

You will be workig with the same cars dataset as in TP1.  
Each group shall be using the same `brands` as in TP1.


In [2]:
# Load dataset and extract our part
import pandas as pd

# Reading csv file
autos = pd.read_csv('autos.csv',encoding='latin-1')

# Extracting the relevant part for our group
only_specific_brands = autos.brand.isin(['renault', 'peugeot', 'skoda', 'citroen', 'ford'])
autos = autos[only_specific_brands]

In [3]:
autos.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage,fast_sale
2,11400.0,limousine,2010.0,manuell,175.0,mondeo,125000.0,diesel,ford,nein,False
4,4100.0,kleinwagen,2009.0,manuell,68.0,1_reihe,90000.0,benzin,peugeot,nein,False
6,888.0,kombi,2000.0,manuell,115.0,mondeo,150000.0,benzin,ford,nein,True
7,13700.0,bus,2012.0,manuell,86.0,roomster,5000.0,benzin,skoda,nein,True
9,4299.0,kleinwagen,2010.0,manuell,75.0,2_reihe,125000.0,benzin,peugeot,nein,False


# Data preprocessing

Remember that after loading the dataset, there are several preprocessing steps you need to do before trainign the algorithm.
If you are not sure what these are, see *Course 8 - 02 Hyper-parameter tuning*.

When writing the code, **put short comments explaining what the pre-processing steps are and why you need to do them**.

### First we are changing the column names:

- We changed the style of writing the column names from camelcase to snakecase. This change will make column names easier to read, and we won't have to remember which letter is capitalized.
- We reworded some of the column names in order for them to be more descriptive and obvious.

In [4]:
# Changing the column names
autos.columns = ['price', 'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model', 
                 'kilometer', 'fuel_type', 'brand', 'unrepaired_damage', 'fast_sale']
autos.head()

,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage,fast_sale
2,11400.0,limousine,2010.0,manuell,175.0,mondeo,125000.0,diesel,ford,nein,False
4,4100.0,kleinwagen,2009.0,manuell,68.0,1_reihe,90000.0,benzin,peugeot,nein,False
6,888.0,kombi,2000.0,manuell,115.0,mondeo,150000.0,benzin,ford,nein,True
7,13700.0,bus,2012.0,manuell,86.0,roomster,5000.0,benzin,skoda,nein,True
9,4299.0,kleinwagen,2010.0,manuell,75.0,2_reihe,125000.0,benzin,peugeot,nein,False


### Next, we are going to convert some columns data type:

- We are changing data type in order to facilitate the data processing (classifying the data in numeric or categorical)
- We are converting column 'unrepaired_damage' from object to boolean type
- We are converting column 'registration_year' from float to int.

In [5]:
# Converting column 'unrepaired_damage' from object to boolean type
autos['unrepaired_damage'] = (autos['unrepaired_damage']
                 .str.replace('nein','')
                 .str.replace('ja','True')
                  .astype(bool)
                 )

In [6]:
# Converting column 'registration_year' from float to int.
autos['registration_year'] = (autos['registration_year'].astype(int))

In [7]:
autos.info()
autos.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28401 entries, 2 to 166073
Data columns (total 11 columns):
price                28401 non-null float64
vehicle_type         28401 non-null object
registration_year    28401 non-null int64
gearbox              28401 non-null object
power_ps             28401 non-null float64
model                28401 non-null object
kilometer            28401 non-null float64
fuel_type            28401 non-null object
brand                28401 non-null object
unrepaired_damage    28401 non-null bool
fast_sale            28401 non-null bool
dtypes: bool(2), float64(3), int64(1), object(5)
memory usage: 2.2+ MB


,price,registration_year,power_ps,kilometer
count,28401.000000,28401.000000,28401.000000,28401.000000
mean,4177.515017,2003.965565,102.402979,121472.307313
std,4680.629533,5.872460,40.664873,39816.529262
min,1.000000,1923.000000,2.000000,5000.000000
25%,1199.000000,2001.000000,75.000000,100000.000000
50%,2500.000000,2004.000000,101.000000,150000.000000
75%,5400.000000,2008.000000,122.000000,150000.000000
max,73500.000000,2016.000000,952.000000,150000.000000


### Next, we are goint to decide what is output (target) and what is input:
- We do this because ...
- How we do it ...

In [8]:
# output dataframe
out_autos = autos['fast_sale']
# input dataframe
in_autos = autos.iloc[:,[0,1,2,3,4,5,6,7,8,9]]

print('Inputs')
display(in_autos.head())
print('Outputs')
display(out_autos.head())

Inputs


,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage
2,11400.0,limousine,2010,manuell,175.0,mondeo,125000.0,diesel,ford,False
4,4100.0,kleinwagen,2009,manuell,68.0,1_reihe,90000.0,benzin,peugeot,False
6,888.0,kombi,2000,manuell,115.0,mondeo,150000.0,benzin,ford,False
7,13700.0,bus,2012,manuell,86.0,roomster,5000.0,benzin,skoda,False
9,4299.0,kleinwagen,2010,manuell,75.0,2_reihe,125000.0,benzin,peugeot,False


Outputs


2    False
4    False
6     True
7     True
9    False
Name: fast_sale, dtype: bool

### Next, we decide which is numerical and which is categorical data and how to treat in the following

- We do this because ...
- How we do it ...

In [9]:
print(in_autos.dtypes)
# DONE IN PREVIOUS STEP
# CHECK IF TYPES ARE CORRECT

price                float64
vehicle_type          object
registration_year      int64
gearbox               object
power_ps             float64
model                 object
kilometer            float64
fuel_type             object
brand                 object
unrepaired_damage       bool
dtype: object


### Next, we convert categorical to numeric (one-hot encoding)

- The algorithms we use (except the deciscion tree) cannot operate on label data directly. They require all input variables and output variables to be numeric. Therefore we need to convert categorical data to a numerical form.
- Since no ordinal relationship exists in the categorical variables in our dataset exists (and we don't want to allow the model to assume a natural ordering between categories), we use the one-hot encoding. 

In [10]:
print('Original input data')
print('-------------------')
display(in_autos.head())

# Create one-hot encoding
in_long = pd.get_dummies(in_autos)
print('\n Numerical input data')
print('-------------------')
display(in_long.head())

Original input data
-------------------


,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage
2,11400.0,limousine,2010,manuell,175.0,mondeo,125000.0,diesel,ford,False
4,4100.0,kleinwagen,2009,manuell,68.0,1_reihe,90000.0,benzin,peugeot,False
6,888.0,kombi,2000,manuell,115.0,mondeo,150000.0,benzin,ford,False
7,13700.0,bus,2012,manuell,86.0,roomster,5000.0,benzin,skoda,False
9,4299.0,kleinwagen,2010,manuell,75.0,2_reihe,125000.0,benzin,peugeot,False



 Numerical input data
-------------------


,price,registration_year,power_ps,kilometer,unrepaired_damage,vehicle_type_andere,vehicle_type_bus,vehicle_type_cabrio,vehicle_type_coupe,vehicle_type_kleinwagen,...,fuel_type_cng,fuel_type_diesel,fuel_type_elektro,fuel_type_hybrid,fuel_type_lpg,brand_citroen,brand_ford,brand_peugeot,brand_renault,brand_skoda
2,11400.0,2010,175.0,125000.0,False,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,4100.0,2009,68.0,90000.0,False,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,888.0,2000,115.0,150000.0,False,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,13700.0,2012,86.0,5000.0,False,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,4299.0,2010,75.0,125000.0,False,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


### Next, we normalize (bring all variables to same scale [0,1])

-  We need to normalize the data to ensure that all sources are treated equally, and that data-availability bias (and its corresponding misrepresentation of the data universe) is reduced.

In [12]:
# maximum of each attribute
maxs = in_long.max(axis=0) # max accross rows (axis=0)

print('Maximums')
print('----------')
print(maxs)

# minimum of each attribute
mins = in_long.min(axis=0) # max accross rows (axis=0)

print('\n Minimums')
print('----------')
print(mins)

# min-max normalization
norm_in = (in_long - mins) / (maxs-mins)

# new maximum of each attribute
maxs = norm_in.max(axis=0) # max accross rows (axis=0)

print('\n New maximums')
print('----------')
print(maxs)

# new minimum of each attribute
mins = norm_in.min(axis=0) # max accross rows (axis=0)

print('\n New minimums')
print('----------')
print(mins)

Maximums
----------
price                       73500.0
registration_year            2016.0
power_ps                      952.0
kilometer                  150000.0
unrepaired_damage               1.0
vehicle_type_andere             1.0
vehicle_type_bus                1.0
vehicle_type_cabrio             1.0
vehicle_type_coupe              1.0
vehicle_type_kleinwagen         1.0
vehicle_type_kombi              1.0
vehicle_type_limousine          1.0
vehicle_type_suv                1.0
gearbox_automatik               1.0
gearbox_manuell                 1.0
model_1_reihe                   1.0
model_2_reihe                   1.0
model_3_reihe                   1.0
model_4_reihe                   1.0
model_5_reihe                   1.0
model_andere                    1.0
model_b_max                     1.0
model_berlingo                  1.0
model_c1                        1.0
model_c2                        1.0
model_c3                        1.0
model_c4                        1.0
model_c5

### As of this step, no more use of original data and only use preprocessed

In our case above:`out_autos` and `norm_in`

# Prepare for model evaluation and hyper-parameter tuning

### Data splits for model evaluation (training and testing)

You will need to write the code splitting the data to training set (used for model learning and hyper-parameter tuning) and test set used for final model evaluation (test error).

Here, you can choose to **use either 5-folds cross-validation or 5 time repeated hold-out method.**

**Tell us what your choice is and why**. Both choices are good, we just want to know that you understand the differences and you have thought about them.

Remember that in the end this procedure will be used for all your algorithms and that these should work over the same train/test splits. You can make sure this will be the case by fixing the seed for the random sample generation.

### 5-folds cross-validation
We chose 5-folds cross-validation because it generally results in a less biased or less optimistic estimate of the model skill than other methods, such as a simple train/test split (hold-out).

In [14]:
from sklearn.model_selection import KFold


n_splits = 5
random_state = 1000
k_fold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True) 

for train,test in k_fold.split(norm_in,out_autos):
    print(f'train: {train} | test: {test}')

print('Test outputs')
display(out_autos.iloc[test].head())
print('Test inputs')
display(norm_in.iloc[test].head())  

train: [    1     2     5 ... 28397 28398 28400] | test: [    0     3     4 ... 28390 28393 28399]
train: [    0     1     2 ... 28397 28398 28399] | test: [    7    14    26 ... 28391 28395 28400]
train: [    0     3     4 ... 28398 28399 28400] | test: [    1     2    11 ... 28387 28388 28394]
train: [    0     1     2 ... 28395 28399 28400] | test: [    5    10    16 ... 28396 28397 28398]
train: [    0     1     2 ... 28398 28399 28400] | test: [   12    15    22 ... 28363 28381 28385]
Test outputs


62      True
78     False
108     True
143     True
152    False
Name: fast_sale, dtype: bool

Test inputs


,price,registration_year,power_ps,kilometer,unrepaired_damage,vehicle_type_andere,vehicle_type_bus,vehicle_type_cabrio,vehicle_type_coupe,vehicle_type_kleinwagen,...,fuel_type_cng,fuel_type_diesel,fuel_type_elektro,fuel_type_hybrid,fuel_type_lpg,brand_citroen,brand_ford,brand_peugeot,brand_renault,brand_skoda
62,0.008150,0.784946,0.076842,0.586207,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
78,0.029919,0.881720,0.135789,1.000000,0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
108,0.095226,0.935484,0.094737,0.827586,0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
143,0.019034,0.881720,0.155789,1.000000,1,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
152,0.000122,0.849462,0.044211,0.827586,1,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Data splits for hyper-parameter tuning

Here we want you to **use 3-folds inner cross validation**.

You will need to write the code to split each of the training sets above to train/validation accoridng to the 3-fold cross-validation strategy.

### 3-folds inner cross validation 

In [15]:
from sklearn.model_selection import KFold


n_splits = 3
random_state = 1000
k_fold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True) 

for train,test in k_fold.split(norm_in,out_autos):
    print(f'train: {train} | test: {test}')

train: [    1     2     5 ... 28397 28398 28400] | test: [    0     3     4 ... 28393 28395 28399]
train: [    0     3     4 ... 28397 28398 28399] | test: [    1     2    11 ... 28388 28394 28400]
train: [    0     1     2 ... 28395 28399 28400] | test: [    5    10    12 ... 28396 28397 28398]


### Generalization accuracy

You will also need to prepere the code that will use the trained models to produce predictions for the test instances, calculate the accuracy of over each test set, and calculate the final average accuracy over all the test instances (estimate of generalization accuracy).

In [ ]:
# What ?

# Train and test nearest neigbour model

Once you have the general procedure in place, train the nearest neigbour model.

### Hyper-parameter search

Hyper-parameter in nearest neighbour algorithm is the number of neighbours to use.
We want you to try at least 5 different values. **Tell us which values you decide to try.** (There is no Why questoin here.)

Remember that for choosing the best hyper-parameter value, you use the the inner cross validation and the best hyper-parameter is the one with the highest average accuracy over the validation sets.

### Model lerning and test accuracy

Once you have the best value of the hyper-parameter, you use it to **retrain** the model over the merged train+validation (you do this 5 times, see above *Data splits for model evaluation*). You then use this **retrained** model to get the final test accuracy.

For each of the test samples (there should be 5, see above), report the test accuracy and the corresponding hyper-parameter setting (the one chosen as best for this specific split).

Are the hyper-parameter parameters the same for all the test sets? **Discuss** if you think this is  normal or not, why it happens and if it creates some difficulties for interpreting the model. **There is no single correct answer here!** We want to see that you undertand the procedure and that you use your brain.



In [ ]:
def nearest_neigbour_classifier():
    # CODE GOES HERE
    pass

In [ ]:
# here we calculate accuracies for the diferent 
# hyper-parameter in nearest neignbour algorithm

In [ ]:
nn_clf_accuracies = None

# Calculate average accuracy for Default Dlassifer
train_accuracy, test_accuracy = calculate_average_accuracy(nn_clf_accuracies)

print('Average train accuracy: ',train_accuracy)
print('Average test accuracy: ',test_accuracy) 


# Train and test decision tree

Use the same general procedure to train a decision tree.

Hyper-parameters for decision trees are the pre-prunning criteria such as maximum number of leafs (see *Course 5 - 02 Decision tree prunning*). 
Pick one of these and use at least 5 different values. **Tell us which one you pick and what values you are using.**

Calculate and report the test accuracies together with their corresponding heper-parameter values. (No more comments needed here.)

In [ ]:
from sklearn import tree


def deciscion_tree_classifier(out_train, out_test, in_train, in_test, max_leaf_nodes=15):
    # instantiate decition tree classifier
    dt_clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)

    # learn tree on train data
    dt_clf = dt_clf.fit(in_train,out_train)

    # make predictions for train and test set
    pred_train = dt_clf.predict(in_train)
    pred_test =  dt_clf.predict(in_test)

    # calculate train accuracy
    train_accuracy = calculate_pred_accuracy(pred_train, out_train)

    # calculate test accuracys
    test_accuracy = calculate_pred_accuracy(pred_test, out_test)

    return (train_accuracy, test_accuracy, dt_clf)

In [ ]:
import random


random.seed(10000)
dt_clf_accuracies = list()
dt_clf = None

for _ in range(0, 10):
    out_train, out_test, in_train, in_test = split_data(
                                                    df=autos, 
                                                    target_var='fast_sale', 
                                                    in_columns=[0,1,2,3,4,5,6,7,8], 
                                                    train_size=2/3, 
                                                    test_size=1/3,
                                                    random_seed=random.randint(1, 10000)
                                             )
    train_acc, test_acc, dt_clf = deciscion_tree_classifier(out_train, out_test, in_train, in_test)
    accuracy_couple = (train_acc, test_acc)
    dt_clf_accuracies.append(accuracy_couple)

In [ ]:
# Calculate average accuracy for Deciscion Tree
train_accuracy, test_accuracy = calculate_average_accuracy(dt_clf_accuracies)

print('Average train accuracy: ',train_accuracy)
print('Average test accuracy: ',test_accuracy) 

# Train and test default classifier

Default classifier has no hyper-parameters, so you can skip the inner-cross validatoin procedure.

Calculate and report the test accuracies for the 5 test sets from the part *Data splits for model evaluation*

In [ ]:
from sklearn.model_selection import train_test_split


def default_classifier(out_train, out_test):
    # get counts per class
    class_counts = out_train.value_counts()

    majority_class = class_counts.idxmax()

    # make predictions for train and test set
    pred_train = out_train.copy()
    pred_train[:] = majority_class

    pred_test = out_test.copy()
    pred_test[:] = majority_class

    # calculate train accuracy
    train_accuracy = calculate_pred_accuracy(pred_train, out_train)

    # calculate test accuracy
    test_accuracy = calculate_pred_accuracy(pred_test, out_test)
    
    return (train_accuracy, test_accuracy)

In [ ]:
import random


random.seed(10000)
default_clf_accuracies = list()

for _ in range(0, 10):
    out_train, out_test, in_train, in_test = split_data(
                                                    df=autos, 
                                                    target_var='fast_sale', 
                                                    in_columns=[0,1,2,3,4,5,6,7,8], 
                                                    train_size=2/3, 
                                                    test_size=1/3,
                                                    random_seed=random.randint(1, 10000)
                                             )
    default_clf_accuracies.append(default_classifier(out_train, out_test))

In [ ]:
# Calculate average accuracy for Default Dlassifer
train_accuracy, test_accuracy = calculate_average_accuracy(default_clf_accuracies)

print('Average train accuracy: ',train_accuracy)
print('Average test accuracy: ',test_accuracy) 

#  /!\ THIS IS THE MOST IMPORTANT PART /!\

# Compare models

Once you have all your test accuracies for the nearest neighbour, decision trees and default classifier, calculate the estimated generalization accuracy of each (the everage accuracy accross the test sets).

Is any of the algorithms peforming better than the other two? **Discuss, comment.**

## Use the McNemar test 

Use the McNemar test to verify whether the differences in the generalization accuracy are significant. 

In McNemar you can always compare only two algorithms. Do all the pair-wise comparisons, present and **explain** the results. Are these what you would expect?

# Comparing the models

### Write an observation for each algorithm and compare performance
#### ==>
### Explain WHY some algorithms are better than the others
#### ==>

# The McNemar test

### Nearest neighbour vs Decision trees 
#### Result
#### Explaination
### Nearest neighbour vs Default classifier
#### Result
#### Explaination
### Decision trees vs Default classifier
#### Result
#### Explaination